In [1]:
import numpy as np 
import pandas as pd
import os
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input as pre_densenet
from eval_utils import *

SRC = "/home/ubuntu/crack/Origin-Unlabeled"
STEP = 500

if os.path.exists("save"):
    shutil.rmtree("save")
os.mkdir("save")

full_list = sorted([file for file in os.listdir(SRC) if file.endswith("JPG") and not file.startswith(".")])
index_step = np.arange(0, len(full_list), STEP)

for i in range(len(index_step)):
    start = index_step[i]
    if i == len(index_step) - 1:
        end = None
    else:
        end = index_step[i + 1]
    image_list = full_list[start:end]
    print("Now Processing Images {} to {} ".format(image_list[0], image_list[-1]), end="\r")
    process_image(SRC, image_list)
    images = load_image((224,224))
    model = DenseNet201()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    features = extract_in_batch(10, images, model, pre_densenet)
    model = load_model('model/binary_TL_DenseNet201.h5')
    y_pred_bi = np.where(model.predict(features) > 0.5, 1, 0).flatten()
    y_pred_bi = np.array(decode(y_pred_bi))
    model = load_model('model/multi_TL_DenseNet201.h5')
    y_pred_mul = (np.argmax(model.predict(features), axis=1) + 2).flatten()
    y_pred_mul = np.array(decode(y_pred_mul))
    y_pred_mul[y_pred_bi == "none"] = "none"
    df = pd.DataFrame(np.array(y_pred_mul).reshape((-1, 3)))
    df.columns = ["crack_1", "crack_2", "crack_3"]
    df.index = image_list
    df.index.name = "image_name"
    df.to_csv("save/{}.csv".format(str(i).zfill(3)))
    
csv_list = sorted([file for file in os.listdir("save") if file.endswith("csv")])
data = pd.concat([pd.read_csv("save/" + file, index_col=0) for file in csv_list])
data.to_csv("prediction.csv")

shutil.rmtree("save")
shutil.rmtree("temp")

Extension horovod.torch has not been built: /home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2021-09-09 12:48:14.465 ip-172-31-5-51:17229 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-09-09 12:48:14.486 ip-172-31-5-51:17229 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
